In [31]:
import pandas as pd

In [32]:
from pathlib import Path

base = Path(r"J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Monthly Operating Statements")

year_dirs = [p for p in base.iterdir() if p.is_dir() and p.name.isdigit()]
if not year_dirs:
    raise FileNotFoundError(f"No year folders found under {base!s}")
latest_year_dir = max(year_dirs, key=lambda p: int(p.name))

pattern_files = list(
    latest_year_dir.glob("Cumulative Report - Operating Statements - *.xlsx")
)
xlsx_files = pattern_files or list(latest_year_dir.glob("*.xlsx"))
if not xlsx_files:
    raise FileNotFoundError(f"No .xlsx files found in {latest_year_dir!s}")

latest_report = max(xlsx_files, key=lambda p: p.stat().st_mtime)

report_path = latest_report
print(report_path)

dor = pd.read_excel(report_path, sheet_name="Summary - DC only", skiprows=6)


dor_end_date = report_path.stem.split(" - ")[-2]

dor_end_date = (
    pd.to_datetime(dor_end_date, format="%m%y", errors="raise")
    .to_period("M")
    .to_timestamp("M")
    .date()
)

print(dor_end_date)

J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Monthly Operating Statements\2025\Cumulative Report - Operating Statements - 1025 - Hard Coded.xlsx
2025-10-31


In [33]:
print(dor.dtypes)

Project ID                                                               object
OL Project Status                                                        object
Start Date                                                       datetime64[ns]
End Date                                                         datetime64[ns]
Short "Project" Title                                                    object
Principal Investigator                                                   object
Funder Type                                                              object
Invoice Type                                                             object
Total PFSR Direct Budget                                                float64
Current Month's\nDirect Expenses                                          int64
Total Direct Expenses                                                   float64
Total PFSR Direct Budget \nvs Total Direct\nExpenses Variance           float64
Total Direct Payments Received          

In [34]:
# drop Unnamed: 0 only if it exists
# if "Unnamed: 0" in dor.columns:
#     dor = dor.drop(columns=["Unnamed: 0"])

# # desired columns (use canonical names)
# desired_cols = [
#     "Project ID",
#     "Project Title",
#     "Program Area",
#     "Funder Type",
#     "Principal Investigator (PI)",
#     "Award Term Start Date",
#     "Project Status",
#     "Total Cash Receipts",
#     "Total Personnel",
#     "Total Contractual/ Outside Services Costs",
#     "Total Non-Personnel",
#     "Total Cost",
# ]

# # normalize helper to match columns ignoring whitespace/newlines/case
# normalize = lambda s: "".join(s.split()).lower() if isinstance(s, str) else s
# col_map = {normalize(c): c for c in dor.columns}

# # build selected column list from available columns (skip missing ones)
# selected = []
# missing = []
# for c in desired_cols:
#     key = normalize(c)
#     if key in col_map:
#         selected.append(col_map[key])
#     else:
#         missing.append(c)

# if missing:
#     print(
#         f"Warning: these desired columns were not found and will be skipped: {missing}"
#     )

# # subset dataframe to the selected (available) columns
# dor = dor[selected]

# print(dor.dtypes)

dor.to_excel(
    "C:\\Users\\O304312\\OneDrive - Kaiser Permanente\\Documents\\Tableau Dashboards\\New Financial Snapshot\\Data\\DOR Data Preprocessed.xlsx",
    index=False,
)

In [35]:
txn_base = Path(r"J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Transaction Detail")
pattern = "CTP Transaction Detail *.xlsx"
matches = list(txn_base.glob(pattern))

if matches:
    ctp_path = max(matches, key=lambda p: p.stat().st_mtime)
else:

    fallback = txn_base / "CTP Transaction Detail 103125.xlsx"
    if fallback.exists():
        ctp_path = fallback
    else:
        raise FileNotFoundError(
            f"No files matching {pattern!s} and fallback {fallback!s} not found in {txn_base!s}"
        )

print("Loading:", ctp_path)

ctp_hours = pd.read_excel(ctp_path, sheet_name="Hours", header=[8, 9, 10, 11])


def tidy(col):
    parts = [
        str(x).strip()
        for x in col
        if str(x).strip() not in {"nan", ""} and not str(x).startswith("Unnamed")
    ]
    return "_".join(parts).strip("_")


ctp_hours.columns = [tidy(col) for col in ctp_hours.columns]

ctp_hours = ctp_hours.rename(columns={"Project": "Project ID"})

mask = ctp_hours["Project ID"].astype(str).str.strip().str.lower().str.endswith("total")
ctp_hours = ctp_hours[mask].copy()
ctp_hours.reset_index(drop=True, inplace=True)

rng_mask = (
    ctp_hours["Project ID"].astype(str).str.match(r"^(RNG\d+)\s+Total$", na=False)
)
ctp_hours.loc[rng_mask, "Project ID"] = (
    ctp_hours.loc[rng_mask, "Project ID"]
    .astype(str)
    .str.replace(r"^(RNG\d+)\s+Total$", r"\1", regex=True)
    .str.strip()
)


print("Columns:", ctp_hours.columns.tolist())

print("Loaded dataframe shape:", ctp_hours.shape)

ctp_hours.to_excel(
    "C:\\Users\\O304312\\OneDrive - Kaiser Permanente\\Documents\\Tableau Dashboards\\New Financial Snapshot\\Data\\DOR Personnel.xlsx",
    index=False,
)

Loading: J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Transaction Detail\CTP Transaction Detail 103125.xlsx
Columns: ['Project ID', 'NUID', 'Name', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025_Qtr1_Jan', '2025_Qtr1_Feb', '2025_Qtr1_Mar', '2025_Qtr2_Apr', '2025_Qtr2_May', '2025_Qtr2_Jun', '2025_Qtr3_Jul', '2025_Qtr3_Aug', '2025_Qtr3_Sep', '2025_Qtr4_Oct', '2025 Total', 'Grand Total']
Loaded dataframe shape: (214, 26)


In [36]:
import re
import json

vp_folder = Path("J:\\VIEWPOINT\\SiteStudyDetails_Response")

pattern = re.compile(r"^\d{4}-\d{2}-\d{2}-SiteStudyDetails\.json$")

matches = [p for p in vp_folder.iterdir() if p.is_file() and pattern.match(p.name)]
if not matches:
    raise FileNotFoundError(f"No SiteStudyDetails json files found in {vp_folder!s}")

latest_json = max(matches, key=lambda p: p.stat().st_mtime)
print("Loading:", latest_json)

with latest_json.open("r", encoding="utf-8") as f:
    site_details = json.load(f)

try:
    ss_df = pd.json_normalize(site_details)
    print("Converted to DataFrame with shape:", ss_df.shape)
except Exception:
    ss_df = None
    print("JSON loaded into 'site_details' (not converted to DataFrame).")

vp_study_details = ss_df.copy(deep=True)

Loading: J:\VIEWPOINT\SiteStudyDetails_Response\2025-12-05-SiteStudyDetails.json
Converted to DataFrame with shape: (1078, 40)


In [37]:
vp_accountables_folder = Path("J:\\VIEWPOINT\\Accountables_Response")

pattern = re.compile(r"^\d{4}-\d{2}-\d{2}-Accountables\.json$")

matches = [
    p for p in vp_accountables_folder.iterdir() if p.is_file() and pattern.match(p.name)
]
if not matches:
    raise FileNotFoundError(
        f"No SiteStudyDetails json files found in {vp_accountables_folder!s}"
    )

latest_json = max(matches, key=lambda p: p.stat().st_mtime)
print("Loading:", latest_json)

with latest_json.open("r", encoding="utf-8") as f:
    site_details = json.load(f)

try:
    account = pd.json_normalize(site_details)
    print("Converted to DataFrame with shape:", account.shape)
except Exception:
    account = None
    print("JSON loaded into 'site_details' (not converted to DataFrame).")

vp_accountables = account.copy(deep=True)

Loading: J:\VIEWPOINT\Accountables_Response\2025-12-05-Accountables.json
Converted to DataFrame with shape: (12089, 38)


In [38]:
vp_accountables = vp_accountables.merge(
    vp_study_details[["site_study_service_line", "network_study_uuid"]],
    on="network_study_uuid",
    how="left",
)

In [39]:
# parse completion_date safely and compare against a Timestamp cutoff
vp_accountables["completion_date_parsed"] = pd.to_datetime(
    vp_accountables["completion_date"], errors="coerce"
)

# ensure dor_end_date is a pandas Timestamp (not a python date)
cutoff = pd.to_datetime(dor_end_date)

vp_accountables = vp_accountables[
    vp_accountables["completion_date_parsed"] < cutoff
].copy()

# remove helper column
vp_accountables.drop(columns=["completion_date_parsed"], inplace=True)

In [40]:
vp_accountables = vp_accountables[["site_study_service_line", "amount"]]
vp_accountables = vp_accountables.groupby("site_study_service_line").sum().reset_index()

In [41]:
# Remove control characters that cause openpyxl IllegalCharacterError
# keep common whitespace (\t, \n, \r); remove other < 0x20 controls
ctl_re = re.compile(r"[\x00-\x08\x0b\x0c\x0e-\x1f]")
for col in vp_accountables.select_dtypes(include=["object"]).columns:
    mask = vp_accountables[col].notna()
    vp_accountables.loc[mask, col] = (
        vp_accountables.loc[mask, col].astype(str).map(lambda s: ctl_re.sub("", s))
    )

vp_accountables.to_excel(
    r"C:\Users\O304312\OneDrive - Kaiser Permanente\Documents\Tableau Dashboards\New Financial Snapshot\Data\Viewpoint Accountables.xlsx",
    index=False,
)

In [42]:
vp_study_details.to_excel(
    r"C:\Users\O304312\OneDrive - Kaiser Permanente\Documents\Tableau Dashboards\New Financial Snapshot\Data\Viewpoint Site Study Details.xlsx",
    index=False,
)

In [43]:
dor["ID"] = dor["Project Title"]

program_col = next(
    (c for c in dor.columns if normalize(c) == normalize("Program Area")), None
)

if program_col is not None:

    def _remove_prog(id_val, prog_val):
        if pd.isna(id_val) or pd.isna(prog_val):
            return id_val
        id_s = str(id_val)
        prog_s = str(prog_val).strip()
        if not prog_s:
            return id_s
        out = re.sub(re.escape(prog_s), "", id_s, flags=re.IGNORECASE)
        out = re.sub(r"[\-\–\—:;\/]+", " ", out)
        out = " ".join(out.split()).strip()
        return out

    dor["ID"] = dor.apply(lambda r: _remove_prog(r["ID"], r[program_col]), axis=1)
else:
    print("Warning: 'Program Area' column not found in dor; ID left unchanged.")

KeyError: 'Project Title'

In [ ]:
vp_set = set(
    vp_study_details["site_study_service_line"]
    .dropna()
    .astype(str)
    .map(lambda s: "".join(s.split()).casefold())
)

dor_norm = dor["Project ID"].astype(str).map(lambda s: "".join(s.split()).casefold())

mask_missing = ~dor_norm.isin(vp_set)

title_col = next(
    (c for c in dor.columns if normalize(c) == normalize("Project Title")), None
)

cols = ["Project ID"]
if title_col:
    cols.append(title_col)

missing_df = dor.loc[mask_missing, cols].drop_duplicates().copy()

vp_map = {}
for sl, sid in (
    vp_study_details[["site_study_service_line", "site_study_id"]]
    .dropna(subset=["site_study_service_line"])
    .itertuples(index=False)
):
    key = "".join(str(sl).split()).casefold()
    if key and pd.notna(sid):
        # keep first occurrence
        vp_map.setdefault(key, sid)

missing_df["site_study_id"] = (
    missing_df["Project ID"]
    .astype(str)
    .map(lambda s: "".join(s.split()).casefold())
    .map(vp_map)
)

if title_col:
    missing_df["site_study_id"] = missing_df["site_study_id"].fillna(
        missing_df[title_col]
    )

out_cols = ["site_study_id", "Project ID"]
if title_col:
    out_cols.append(title_col)

missing_out = missing_df.loc[:, out_cols]

print(
    f"{len(missing_out)} Project ID(s) not found in site_study_service_line (saved to file)"
)

out_path = r"C:\Users\O304312\OneDrive - Kaiser Permanente\Documents\Tableau Dashboards\New Financial Snapshot\Data\VP Missing RNG Numbers.xlsx"
missing_out.to_excel(out_path, index=False)

153 Project ID(s) not found in site_study_service_line (saved to file)
